# Multi-device time synchronization tutorial (for Aria Gen2)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from typing import Iterator, Any
import os

from projectaria_tools.core import data_provider
from projectaria_tools.core.data_provider import VrsMetadata, MetadataTimeSyncMode
from projectaria_tools.core.sensor_data import (
    SensorData,
    ImageData,
    TimeDomain,
    TimeQueryOptions,
    TimeSyncMode,
)
from projectaria_tools.core.stream_id import StreamId

## Download time-synced recordings
Download the `gen2_timesync_server.vrs` and `gen2_timesync_client.vrs` files from Google Drive, and modify the following code to point to their download path

In [ ]:
# Create server and client data provider from recordings
data_folder = os.path.join(os.path.expanduser("~"), "Downloads", "06_04_PAT_Gen2_External_BugBash")

server_recording = os.path.join(data_folder, "gen2_timesync_server.vrs")
server_data_provider = data_provider.create_vrs_data_provider(server_recording)

client_recording = os.path.join(data_folder, "gen2_timesync_client.vrs")
client_data_provider = data_provider.create_vrs_data_provider(client_recording)

## Find a valid timestamp on server time domain
Because server and client devices could start at different time, therefore we first locate a timestamp that both devices are guaranteed to have valid data. 

In [ ]:
# Fetch a given RGB frame from the server device
rgb_stream_id = server_data_provider.get_stream_id_from_label("camera-rgb")
assert rgb_stream_id is not None, "RGB stream does not exist in server vrs"

# Find the first client timestamp, and convert it to server time.
first_client_timestamp = client_data_provider.get_first_time_ns(rgb_stream_id, TimeDomain.SUBGHZ)

# Pick a server timetamp, which client device is guaranteed to have data
first_server_timestamp = server_data_provider.get_first_time_ns(rgb_stream_id, TimeDomain.DEVICE_TIME)
server_timestamp = max(first_server_timestamp, first_client_timestamp) + int(16.1e9)
print(f"Fetching from server timestamp {server_timestamp}")

### Fetch synced images from both devices

In [ ]:
# Fetch RGB frame from server device. Note that we only need to fetch by DEVICE TIME on server, because it broadcasts DEVICE TIME
server_image_data = server_data_provider.get_image_data_by_time_ns(rgb_stream_id, server_timestamp, TimeDomain.DEVICE_TIME)

# Fetch the corresponding RGB frame from the client device by SUBGHZ time domain.
client_image_data = client_data_provider.get_image_data_by_time_ns(rgb_stream_id, server_timestamp, TimeDomain.SUBGHZ)

# Print the timestamp difference between query and result
timestamp_ns_difference = client_image_data[1].capture_timestamp_ns - client_data_provider.convert_from_synctime_to_device_time_ns(server_timestamp, TimeSyncMode.SUBGHZ)
print(f"Timestamp difference between query and result: {timestamp_ns_difference/1e6} ms")


# Display the fetched RGB frames between server and client
axes = []
fig, axes = plt.subplots(1, 2, figsize=(30, 15))
fig.suptitle('Retrieving time synced image data from server and client')

axes[0].imshow(server_image_data[0].to_numpy_array(), cmap="gray", vmin=0, vmax=255)
axes[0].title.set_text("server image")
axes[0].grid(False)
axes[0].tick_params(left=False, right=False, labelleft=False, labelbottom=False, bottom=False)

axes[1].imshow(client_image_data[0].to_numpy_array(), cmap="gray", vmin=0, vmax=255)
axes[1].title.set_text("client image")
axes[1].grid(False)
axes[1].tick_params(left=False, right=False, labelleft=False, labelbottom=False, bottom=False)

plt.show()